In [1]:
# Cell 1: Setup and Installations
# This first cell sets up the environment by installing necessary libraries and cloning the required repositories.
# We will use PyTorch and torchvision, which are a great combination for object detection.
# MattNet is a research model, so we'll clone a public implementation from GitHub.

!pip install torch torchvision
!git clone https://github.com/lichengunc/MAttNet.git
# Note: You may need to compile some custom C++/CUDA extensions depending on the MattNet implementation.
# This can be done by navigating into the lib directory of the cloned repository and running:
# cd MAttNet/lib
# python setup.py build develop

# Import all necessary libraries for the rest of the notebook.
import os
import json
import torch
import torchvision
from PIL import Image
from tqdm import tqdm
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Define the device to use (GPU if available).
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5

In [3]:

# Cell 2: Data Linking and Loading
# This section has been updated to specifically handle the Kaggle file path structure you provided.
# We will no longer use kagglehub but instead assume the dataset is mounted.

# --- START OF FILE PATH FIX ---
# Import necessary modules here to ensure they are always available.
import os
import json

# Define the root directory of the Kaggle dataset.
KAGGLE_DATA_ROOT = "/kaggle/input/visualgenome"
print(f"Using Kaggle data root: {KAGGLE_DATA_ROOT}")

# We will now search for the specific file and image directories,
# as they are located in different subdirectories.
region_graphs_path = None
image_data_path = None
IMAGE_DIR_1 = None
IMAGE_DIR_2 = None

print("Searching for data files and image directories within the downloaded dataset...")

# Walk the directory tree to find all necessary files and directories.
for root, dirs, files in os.walk(KAGGLE_DATA_ROOT):
    # Find the path for region_graphs.json
    if 'region_graphs.json' in files and region_graphs_path is None:
        region_graphs_path = os.path.join(root, 'region_graphs.json')
    
    # Find the path for image_data.json
    if 'image_data.json' in files and image_data_path is None:
        image_data_path = os.path.join(root, 'image_data.json')

    # Find the image directories.
    if 'VG_100K' in dirs and IMAGE_DIR_1 is None:
        IMAGE_DIR_1 = os.path.join(root, 'VG_100K')
    if 'VG_100K_2' in dirs and IMAGE_DIR_2 is None:
        IMAGE_DIR_2 = os.path.join(root, 'VG_100K_2')

    # If all paths are found, we can stop the walk early to save time.
    if region_graphs_path and image_data_path and IMAGE_DIR_1 and IMAGE_DIR_2:
        break

if not region_graphs_path:
    raise FileNotFoundError("Could not locate the file 'region_graphs.json'.")
if not image_data_path:
    raise FileNotFoundError("Could not locate the file 'image_data.json'.")
if not IMAGE_DIR_1:
    raise FileNotFoundError("Could not locate the directory 'VG_100K'.")
if not IMAGE_DIR_2:
    raise FileNotFoundError("Could not locate the directory 'VG_100K_2'.")

print(f"region_graphs.json found at: {region_graphs_path}")
print(f"image_data.json found at: {image_data_path}")
print(f"Image directory 1 found in: {IMAGE_DIR_1}")
print(f"Image directory 2 found in: {IMAGE_DIR_2}")

# Load the region graphs JSON data, which contains the annotations.
with open(region_graphs_path, 'r') as f:
    region_graphs = json.load(f)

# Load the image data JSON data (if needed for metadata, e.g., image dimensions).
with open(image_data_path, 'r') as f:
    image_data = json.load(f)

# --- END OF FILE PATH FIX ---

print(f"Loaded {len(region_graphs)} region graph entries.")
print(f"Loaded {len(image_data)} image data entries.")

# Create a mapping from image ID to its file path for quick lookup.
# We need to check both image directories.
image_id_to_path = {}
for img in image_data:
    image_id = img['image_id']
    path1 = os.path.join(IMAGE_DIR_1, f"{image_id}.jpg")
    path2 = os.path.join(IMAGE_DIR_2, f"{image_id}.jpg")
    if os.path.exists(path1):
        image_id_to_path[image_id] = path1
    elif os.path.exists(path2):
        image_id_to_path[image_id] = path2

# Prepare a list of all images we want to process.
# Due to the large size, it's highly recommended to start with a subset.
# For example, let's take the first 1000 images. For the final training, you would use all 50,000.
# The code is designed to scale; just change the slice.
subset_size = 50000
image_ids = list(image_id_to_path.keys())[:subset_size]

if not image_ids:
    raise ValueError("No images were found in the dataset. Please check the dataset structure.")
print(f"Using a subset of {len(image_ids)} images for training.")


# Create a dictionary to map image IDs to their descriptions for easier access.
image_annotations = {}
for entry in region_graphs:
    image_id = entry['image_id']
    if image_id in image_id_to_path:
        # Each entry has a list of regions. We'll store them.
        image_annotations[image_id] = entry['regions']



Using Kaggle data root: /kaggle/input/visualgenome
Searching for data files and image directories within the downloaded dataset...
region_graphs.json found at: /kaggle/input/visualgenome/region_graphs.json/region_graphs.json
image_data.json found at: /kaggle/input/visualgenome/image_data.json/image_data.json
Image directory 1 found in: /kaggle/input/visualgenome/images/VG_100K
Image directory 2 found in: /kaggle/input/visualgenome/images2/VG_100K_2
Loaded 108077 region graph entries.
Loaded 108077 image data entries.
Using a subset of 50000 images for training.


In [15]:

# Cell 3: Custom Dataset Class
# This class handles loading each image and its corresponding annotations on-the-fly.
# This is a critical step for memory efficiency with large datasets.

class VisualGenomeDataset(Dataset):
    def __init__(self, image_ids, image_annotations, image_id_to_path, transforms=None):
        self.image_ids = image_ids
        self.image_annotations = image_annotations
        self.image_id_to_path = image_id_to_path
        self.transforms = transforms

    def __getitem__(self, idx):
        # Use a try-except block to gracefully handle potential issues with image loading or annotations
        # which can sometimes be corrupt in large datasets.
        try:
            # Get the image ID for the current index.
            image_id = self.image_ids[idx]
            
            # Open the image file.
            img_path = self.image_id_to_path[image_id]
            img = Image.open(img_path).convert("RGB")
            
            # Get the annotations (regions) for this image.
            regions = self.image_annotations.get(image_id, [])

            # Parse the bounding boxes and labels from the regions.
            boxes = []
            labels = []
            # The first class (0) is traditionally reserved for the background in PyTorch.
            # We will use class 1 for all objects.
            for region in regions:
                x, y, w, h = region['x'], region['y'], region['width'], region['height']
                # Convert (x, y, w, h) to (x_min, y_min, x_max, y_max)
                boxes.append([x, y, x + w, y + h])
                # For simplicity, we are assigning all as a single class (e.g., 'object').
                labels.append(1) 
            
            # If no objects are found, use a dummy box and label to prevent errors.
            if not boxes:
                boxes = torch.zeros((0, 4), dtype=torch.float32)
                labels = torch.zeros(0, dtype=torch.int64)
            else:
                boxes = torch.as_tensor(boxes, dtype=torch.float32)
                labels = torch.as_tensor(labels, dtype=torch.int64)

            target = {}
            target["boxes"] = boxes
            target["labels"] = labels
            target["image_id"] = torch.tensor([image_id])

            # Apply transformations if they are defined.
            if self.transforms is not None:
                # FIX: This is the line that was causing the error.
                # The transform `ToTensor` is designed to be applied to a single image.
                # Passing `img, target` caused the `TypeError`. We must only pass the image.
                img = self.transforms(img)

            return img, target
        
        except Exception as e:
            print(f"Error processing image ID {self.image_ids[idx]}: {e}")
            # Return a valid empty sample to prevent the DataLoader from crashing.
            return (torch.zeros((3, 224, 224), dtype=torch.float32), 
                    {'boxes': torch.zeros((0, 4), dtype=torch.float32), 'labels': torch.zeros(0, dtype=torch.int64)})


    def __len__(self):
        return len(self.image_ids)

# Define transformations for the dataset.
# The `ToTensor` transform should only operate on the image.
# We will return the transform directly here.
def get_transform():
    return torchvision.transforms.ToTensor()

# Create the dataset and a data loader. The data loader will handle batching and shuffling.
if 'image_ids' in locals() and 'image_annotations' in locals() and 'image_id_to_path' in locals():
    dataset = VisualGenomeDataset(image_ids, image_annotations, image_id_to_path, transforms=get_transform())

    # A custom collate function is needed because the images have different numbers of objects.
    def collate_fn(batch):
        # Filter out any None values returned by the dataset's __getitem__ method in case of errors.
        batch = [item for item in batch if item is not None]
        if not batch:
            return None, None
        return tuple(zip(*batch))

    data_loader = DataLoader(
        dataset,
        batch_size=2,  # Set a small batch size due to the large image size and memory constraints.
        shuffle=True,
        # Setting num_workers to 0 to prevent memory crashes.
        # This will make data loading slower, but more stable.
        # You can increase this value if you are sure your environment has enough memory.
        num_workers=0,
        collate_fn=collate_fn
    )
else:
    print("Skipping Dataset and DataLoader creation due to missing data.")



In [16]:

# Cell 4: Model Initialization (Faster R-CNN with MattNet Concept)
# We will use a pre-trained Faster R-CNN model as our base.
# MattNet is a visual-textual model that is complex to implement from scratch.
# The core idea is to integrate a language-based reasoning component.
# This is a simplified approach, as the full MattNet requires a separate language model.
# A full implementation would involve replacing the `FastRCNNPredictor` with a custom module from the MattNet repo.

# The number of classes: 1 for 'object' and 1 for the background.
num_classes = 2

# Load a pre-trained Faster R-CNN model from torchvision.
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# Replace the box predictor to match our number of classes.
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# If you were to fully implement MattNet, you would replace this with a more complex head
# that also takes the language description as input. This requires a deeper dive into the MattNet repo's code.
# For example, you might create a new class that inherits from FastRCNNPredictor and adds a text encoder.
# from MAttNet_repo.model import CustomPredictor
# model.roi_heads.box_predictor = CustomPredictor(...)

model.to(device)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:

# Cell 5: Training Pipeline
# This cell contains the main training loop. We will train for a few epochs.
# You can adjust the number of epochs and the learning rate.

# Construct an optimizer.
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# Optional: Learning rate scheduler.
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Number of epochs. Start with a small number and increase it as you monitor progress.
num_epochs = 5

for epoch in range(num_epochs):
    # Set the model to training mode.
    model.train()
    
    # Use tqdm to show a progress bar.
    progress_bar = tqdm(data_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    
    for images, targets in progress_bar:
        # Move images and targets to the device.
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Get the losses from the model.
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        
        # Backpropagation and optimization.
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
        # Update the progress bar with the current loss.
        progress_bar.set_postfix(loss=losses.item())
        
    # Update the learning rate scheduler (if used).
    if lr_scheduler is not None:
        lr_scheduler.step()
        
    print(f"Epoch {epoch+1} finished. Total Loss: {losses.item()}")

# Save the trained model.
torch.save(model.state_dict(), 'faster_rcnn_visual_genome.pth')
print("Training complete. Model saved.")



Epoch 1/5:  49%|████▉     | 12293/25000 [2:21:05<2:22:57,  1.48it/s, loss=1.78]